<a href="https://colab.research.google.com/github/abelkmathew/healthcare_chatbot/blob/main/Optiml_model.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Disease Prediction ML Model

### Objective

The aim is to classify which disease the patient has, so as to find out which department in the hospital the patient has to be refered to.

### Description

Our ML model is going to be trained with a dataset obtained from Kaggle containing 4921 disease and 132 symptoms that apply to different disease. The user has to input the first symptom, based on this first symptom the ML model makes a First_Prediction and symptoms related to this first prediction is asked to the user. Based on the input given we can get the Final_Prediction and the user can be reffered to a department in the hospital


###Importing Necessary Libraries

In [ ]:
import numpy as np
import csv
import pandas as pd
import os

#IMporting Classifiers
from sklearn.ensemble import RandomForestClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.tree import DecisionTreeClassifier

#impporting Files to find out accuracy
from sklearn.model_selection import train_test_split
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report,confusion_matrix,accuracy_score
from sklearn import preprocessing



import matplotlib.pyplot as plt
from sklearn.preprocessing import StandardScaler
from mpl_toolkits.mplot3d import Axes3D

###Dataset read into Dataframe

In [ ]:

    training = pd.read_csv('training.csv')
    X = training.iloc[:, :-1]
    y = training['prognosis']
    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=20)
    training.head(n=5)

###Data Exploration

In [ ]:
# Distribution graphs (histogram/bar graph) of column data

def plotPerColumnDistribution(df1, nGraphShown, nGraphPerRow):
    nunique = df1.nunique()
    df1 = df1[[col for col in training if nunique[col] > 1 and nunique[col] < 50]] # For displaying purposes, pick columns that have between 1 and 50 unique values
    nRow, nCol = df1.shape
    columnNames = list(df1)
    nGraphRow = (nCol + nGraphPerRow - 1) / nGraphPerRow
    plt.figure(num = None, figsize = (6 * nGraphPerRow, 8 * nGraphRow), dpi = 80, facecolor = 'w', edgecolor = 'k')
    for i in range(min(nCol, nGraphShown)):
        plt.subplot(nGraphRow, nGraphPerRow, i + 1)
        columnDf = training.iloc[:, i]
        if (not np.issubdtype(type(columnDf.iloc[0]), np.number)):
            valueCounts = columnDf.value_counts()
            valueCounts.plot.bar()
        else:
            columnDf.hist()
        plt.ylabel('counts')
        plt.xticks(rotation = 90)
        plt.title(f'{columnNames[i]} (column {i})')
    plt.tight_layout(pad = 1.0, w_pad = 1.0, h_pad = 1.0)
    plt.show()

In [ ]:
# Scatter and density plots

def plotScatterMatrix(df1, plotSize, textSize):
    df1 = df1.select_dtypes(include =[np.number]) # keep only numerical columns
    # Remove rows and columns that would lead to training being singular
    df1 = df1.dropna('columns')
    df1 = df1[[col for col in training if training[col].nunique() > 1]] # keep columns where there are more than 1 unique values
    columnNames = list(training)
    if len(columnNames) > 10: # reduce the number of columns for matrix inversion of kernel density plots
        columnNames = columnNames[:10]
    df1 = df1[columnNames]
    ax = pd.plotting.scatter_matrix(df1, alpha=0.75, figsize=[plotSize, plotSize], diagonal='kde')
    corrs = df1.corr().values
    for i, j in zip(*plt.np.triu_indices_from(ax, k = 1)):
        ax[i, j].annotate('Corr. coef = %.3f' % corrs[i, j], (0.8, 0.2), xycoords='axes fraction', ha='center', va='center', size=textSize)
    plt.suptitle('Scatter and Density Plot')
    plt.show()


In [ ]:
plotPerColumnDistribution(training, 10, 5)

In [ ]:
plotScatterMatrix(training, 20, 10)

In [ ]:
#check for symptom within list of symptoms
def check_pattern(dis_list,inp):
    import re
    pred_list=[]
    ptr=0
    patt = "^" + inp + "$"
    regexp = re.compile(inp)
    for item in dis_list:
        # print(f"comparing {inp} to {item}")
        if regexp.search(item):
            pred_list.append(item)
            # return 1,item
    if(len(pred_list)>0):
        return 1,pred_list
    else:
        return ptr,item

###Model Accuracy

###Model Output

**To build the precision of the model, we utilized three distinctive algorithms which are as per the following**
* Decision Tree algorithm
* Random Forest algorithm
* KNearestNeighbour algorithm
* Naive Bayes algorithm

Decision Tree Algorithm

In [ ]:
rf_clf = DecisionTreeClassifier()
rf_clf.fit(X_train.values, y_train)
list1=training.columns.tolist()
print("Enter the symptom you are experiencing  \n\t\t\t\t\t\t",end="->")
disease_input = input("")
conf,cnf_dis=check_pattern(list1,disease_input)
if conf==1:
  print("searches related to input: ")
  for num,it in enumerate(cnf_dis):
    print(num,")",it)
  if num!=0:
    print(f"Select the one you meant (0 - {num}):  ", end="")
    conf_inp = int(input(""))
  else:
    conf_inp=0
  disease_input=cnf_dis[conf_inp]
symptoms_first=[]
symptoms_first.append(disease_input)
first=rf_clf.predict(symptoms_first)
reduced_data = training.groupby(training['prognosis']).max()
red_cols = reduced_data.columns
symptoms_given = red_cols[reduced_data.loc[first].values[0].nonzero()]
symptoms_given=symptoms_given.tolist()
if (disease_input in symptoms_given):
  symptoms_given.remove(disease_input)
print(symptoms_given)

symptoms_exp=[]
for syms in list(symptoms_given):
  inp=""
  print(syms,"? : ",end='')
  while True:
    inp=input("")
    if(inp=="yes" or inp=="no"):
      break
    else:
      print("provide proper answers i.e. (yes/no) : ",end="")
  if(inp=="yes"):
    symptoms_exp.append(syms)
symptoms_exp.append(disease_input)
print(symptoms_exp)
second_prediction=rf_clf.predict(symptoms_exp)
print(second_prediction[0])

with open('DisDeptClassification.csv',newline='') as csvfile:
                    data=csv.DictReader(csvfile)
                    for row in data:
                        if (row['Disease']==second_prediction[0]):
                            dept=row['Department']
                            print(dept)

Enter the symptom you are experiencing  
						->cold
searches related to input: 
0 ) cold_hands_and_feets
['cold_hands_and_feets']


ValueError: ignored

Random Forest Algorithm

In [ ]:


 rf_clf=RandomForestClassifier()
 rf_clf.fit(X_train.values, y_train)
print("Enter the symptom you are experiencing  \n\t\t\t\t\t\t",end="->")
disease_input = input("")
conf,cnf_dis=check_pattern(list1,disease_input)
if conf==1:
  print("searches related to input: ")
  for num,it in enumerate(cnf_dis):
    print(num,")",it)
  if num!=0:
    print(f"Select the one you meant (0 - {num}):  ", end="")
    conf_inp = int(input(""))
  else:
    conf_inp=0
  disease_input=cnf_dis[conf_inp]
symptoms_first=[]
symptoms_first.append(disease_input)
first=rf_clf.predict([input_vector])
reduced_data = training.groupby(training['prognosis']).max()
red_cols = reduced_data.columns
symptoms_given = red_cols[reduced_data.loc[first].values[0].nonzero()]
symptoms_given=symptoms_given.tolist()
if (disease_input in symptoms_given):
  symptoms_given.remove(disease_input)
print(symptoms_given)

symptoms_exp=[]
for syms in list(symptoms_given):
  inp=""
  print(syms,"? : ",end='')
  while True:
    inp=input("")
    if(inp=="yes" or inp=="no"):
      break
    else:
      print("provide proper answers i.e. (yes/no) : ",end="")
  if(inp=="yes"):
    symptoms_exp.append(syms)
symptoms_exp.append(disease_input)
print(symptoms_exp)
second_prediction=rf_clf.predict([input_vector])
print(second_prediction[0])

with open('DisDeptClassification.csv',newline='') as csvfile:
                    data=csv.DictReader(csvfile)
                    for row in data:
                        if (row['Disease']==second_prediction[0]):
                            dept=row['Department']
                            print(dept)

KNN Algorithm

In [ ]:
rf_clf=KNeighborsClassifier()
rf_clf.fit(X_train.values, y_train)
print("Enter the symptom you are experiencing  \n\t\t\t\t\t\t",end="->")
disease_input = input("")
conf,cnf_dis=check_pattern(list1,disease_input)
if conf==1:
  print("searches related to input: ")
  for num,it in enumerate(cnf_dis):
    print(num,")",it)
  if num!=0:
    print(f"Select the one you meant (0 - {num}):  ", end="")
    conf_inp = int(input(""))
  else:
    conf_inp=0
  disease_input=cnf_dis[conf_inp]
symptoms_first=[]
symptoms_first.append(disease_input)
first=rf_clf.predict([input_vector])
reduced_data = training.groupby(training['prognosis']).max()
red_cols = reduced_data.columns
symptoms_given = red_cols[reduced_data.loc[first].values[0].nonzero()]
symptoms_given=symptoms_given.tolist()
if (disease_input in symptoms_given):
  symptoms_given.remove(disease_input)
print(symptoms_given)

symptoms_exp=[]
for syms in list(symptoms_given):
  inp=""
  print(syms,"? : ",end='')
  while True:
    inp=input("")
    if(inp=="yes" or inp=="no"):
      break
    else:
      print("provide proper answers i.e. (yes/no) : ",end="")
  if(inp=="yes"):
    symptoms_exp.append(syms)
symptoms_exp.append(disease_input)
print(symptoms_exp)
second_prediction=rf_clf.predict([input_vector])
print(second_prediction[0])

with open('DisDeptClassification.csv',newline='') as csvfile:
                    data=csv.DictReader(csvfile)
                    for row in data:
                        if (row['Disease']==second_prediction[0]):
                            dept=row['Department']
                            print(dept)

Naive Bayes Algorithm

In [ ]:

rf_clf=GaussianNB()
rf_clf.fit(X_train.values, y_train)
print("Enter the symptom you are experiencing  \n\t\t\t\t\t\t",end="->")
disease_input = input("")
conf,cnf_dis=check_pattern(list1,disease_input)
if conf==1:
  print("searches related to input: ")
  for num,it in enumerate(cnf_dis):
    print(num,")",it)
  if num!=0:
    print(f"Select the one you meant (0 - {num}):  ", end="")
    conf_inp = int(input(""))
  else:
    conf_inp=0
  disease_input=cnf_dis[conf_inp]
symptoms_first=[]
symptoms_first.append(disease_input)
first=rf_clf.predict([input_vector])
reduced_data = training.groupby(training['prognosis']).max()
red_cols = reduced_data.columns
symptoms_given = red_cols[reduced_data.loc[first].values[0].nonzero()]
symptoms_given=symptoms_given.tolist()
if (disease_input in symptoms_given):
  symptoms_given.remove(disease_input)
print(symptoms_given)

symptoms_exp=[]
for syms in list(symptoms_given):
  inp=""
  print(syms,"? : ",end='')
  while True:
    inp=input("")
    if(inp=="yes" or inp=="no"):
      break
    else:
      print("provide proper answers i.e. (yes/no) : ",end="")
  if(inp=="yes"):
    symptoms_exp.append(syms)
symptoms_exp.append(disease_input)
print(symptoms_exp)
second_prediction=rf_clf.predict([input_vector])
print(second_prediction[0])

with open('DisDeptClassification.csv',newline='') as csvfile:
                    data=csv.DictReader(csvfile)
                    for row in data:
                        if (row['Disease']==second_prediction[0]):
                            dept=row['Department']
                            print(dept)